# Tutorial 4 - Objective magnification

Welcome to the fourth tutorial of this series!
In this notebook, we'll investigate the effect of the objective magnification on the structure function.
This allows us to extend the range of wave vectors that can be analyzed with DDM to lower values.

### Purpose

The goal of this tutorial is to demonstrate the effect of the objective magnification on the results obtained by DDM.
Specifically, we will:

0. Set up the workspace and import the libraries
    1. Set up the workspace.
    2. Give an overview of the sample under investigation.
    3. Import libraries.
1. Compute the structure function of image sequences using DDM.
    1. Load the image sequences and see the effect of different objective magnifications on the input images.
    2. Calculate the corresponding structure functions.
    3. Calculate the azimuthal averages.
2. Fit the azimuthal average to gain insights into the sample's dynamics.
    1. Estimate the parameters.
    2. Fit the data using a diffusive dynamics model.
    3. Observe the effect of the objective magnification on the limits of the technique.
    4. Fit the parameters obtained from the fit to the data to retrieve the diffusion coefficient of the particles.

This tutorial assumes that you have a basic understanding of the features offered by fastDDM. If you need a refresher, consider reviewing the ["zeroth" tutorial](../Tutorial_0-Introduction/tutorial0.ipynb).
It also builds on top of the previous tutorials [1](../Tutorial_1-Particle_sizing/tutorial1.ipynb) and [2](../Tutorial_2-Melt/tutorial2.ipynb).

At the end of the notebook, there will be an optional (starred) section:

* (*): Figures for the objective magnification section of the paper.

## 0. Initial setup

### 0.A. Setting up your workspace

Before we dive into the analysis, make sure you have **fastDDM** and the required scientific analysis, fitting, and visualization packages installed on your machine. Refer to the main [README](../README.md) for installation instructions.

For this tutorial, we'll be using custom microscopy videos acquired for these tutorials.
If you haven't done so already, download the files (the `DATASET` folder) in the directory containing these tutorials.

Your main directory tree should resemble the following:
```
.
├── DATASET
│   └── ...
├── Tutorial_4-Objective_magnification
└── ...
```

### 0.B. Sample description and experimental parameters

The sample under study consists of a dilute solution of polystyrene spheres (PS, nominal diameter 252 nm, Microparticles GmbH) dispersed in a glycerol aqueous solution ($c_{\text{gly}} = 21.5\%$) to prevent sedimentation.
The sample was prepared at a volume fraction $\phi=10^{-3}$ and loaded into a 0.3 $\times$ 3 $\times$ 50 mm glass capillary (Vitrocom).
The experiments were performed at $T \simeq 21^{\circ}\mathrm{C}$ (solvent viscosity $\eta = 1.78 \, \mathrm{mPa \, s}$).

The videos were acquired at 108 and 1 fps for the fast and slow acquisition, respectively, using a Nikon Eclipse Ti2 inverted microscope equipped with a Prime BSI Express CMOS camera (pixel size $6.5 \, \mathrm{µm}$).
Each set of videos was acquired using objectives with different magnification:
* 10$\times$, NA 0.45
* 20$\times$, NA 0.45
* 60$\times$, NA 0.7

The effective pixel size is calculated according to $\delta_{px} = 6.5/m_o \, \mathrm{µm}$, where $m_o$ is the objective magnification.
This gives in the three cases $650$, $325$, and $108 \, \mathrm{nm}$.
The image sequences are composed of 10000 frames each, 256$\times$256 pixels.

### 0.C. Let's get started

Let's import some essential libraries, including `fastDDM`.

In [ ]:
%matplotlib notebook

import gc
import os
import lmfit
import numpy as np

# set numpy print options to limit the length of the output
np.set_printoptions(threshold=0)

# matplotlib
import matplotlib as mpl
import matplotlib.pyplot as plt
import matplotlib.cm as cm
from matplotlib.lines import Line2D
from matplotlib.offsetbox import AnchoredText

# get rid of the 'too many open images' warning
mpl.rcParams.update({'figure.max_open_warning': 0})

# color space helper
def color_space( 
    length: int,
    colormap: cm.colors.LinearSegmentedColormap = cm.viridis,
    vmin: float = 0.2, 
    vmax: float = 1.0
) -> np.ndarray:
    return colormap(np.linspace(vmin, vmax, length))

# fastddm
import fastddm as fd
from fastddm.fit import fit

Here, we define the paths to the videos and the labels.

In [ ]:
# directory containing the videos
main_directory = '../DATASET/'

# high frame rate videos file names (including enclosing folder)
file_names_fast = [
    'PS_252nm_phi_1e-3_10x_NA_0-45/fast.nd2',
    'PS_252nm_phi_1e-3_20x_NA_0-45/fast.nd2',
    'PS_252nm_phi_1e-3_60x_NA_0-7/fast.nd2',
]

# create full paths
file_paths_fast = [os.path.join(main_directory, fn) for fn in file_names_fast]

# low frame rate videos file names (including enclosing folder)
file_names_slow = [
    'PS_252nm_phi_1e-3_10x_NA_0-45/slow.nd2',
    'PS_252nm_phi_1e-3_20x_NA_0-45/slow.nd2',
    'PS_252nm_phi_1e-3_60x_NA_0-7/slow.nd2',
]

# create full paths
file_paths_slow = [os.path.join(main_directory, fn) for fn in file_names_slow]

# list of labels
labels = [
    r'10x, NA$_o$ 0.45',
    r'20x, NA$_o$ 0.45',
    r'60x, NA$_o$ 0.7',
]

## 1. Structure function calculation

### 1.A. Load and visualize images

As usual, we start by inspecting the image sequences.
We show the first frame of the fast videos side by side.

In [ ]:
fig = plt.figure()
gs = fig.add_gridspec(ncols=len(file_paths_fast))
axs = gs.subplots()

# read the images
images = [fd.read_images(fp, seq=[0])[0] for fp in file_paths_fast]
images = np.array(images)

for ax, img, label in zip(axs.flatten(), images, labels):
    ax.imshow(img, cmap='gray')
    
    ax.set_axis_off()
    ax.set_title(label)

fig.tight_layout();

Increasing the objective magnification does what you would expect: it zooms in to see tinier things more clearly.

### 1.B-C. Structure function and azimuthal average calculation

The process here mirrors the one outlined in [tutorial2](../Tutorial_2-Melt/tutorial2.ipynb).
Use the following cell to select the `core` and `mode` you wish to use for the calculation of the structure function.

In [ ]:
### UNCOMMENT THE CORE YOU WISH TO USE ###
#CORE = 'py'
#CORE = 'cpp'
CORE = 'cuda'

### UNCOMMENT THE MODE YOU WISH TO USE ###
#MODE = 'diff'
MODE = 'fft'

Here, we set the experimental parameters of the acquisition.

In [ ]:
# experimental acquisition parameters
magnifications = [10, 20, 60]
pixel_size = [6.5 / mag for mag in magnifications]  # microns per pixel

frame_rate_fast = 108    # frames per second
frame_rate_slow = 1      # frames per second

Finally, we separately compute the structure function and the azimuthal averages for the two acquisitions.
For each pair of azimuthal averages, we compute the melt, which we will keep for the analysis.

In [ ]:
print('--- Calculating structure functions and azimuthal averages... ---')

# initialize list of azimuthal averages
aa = []

for num, (fpathf, fpaths) in enumerate(zip(file_paths_fast, file_paths_slow)):
    # analyze fast acquisition
    # notify user on progress
    print(f'Analyzing fast movie #{num+1} of {len(file_paths_fast)} ...')
    
    # read image sequence
    img_seq = fd.read_images(fpathf)
    
    # compute structure function
    lags = range(1, int(len(img_seq) / 2))              # we analyze delays only up to half the length of the video
    dqt = fd.ddm(img_seq, lags, core=CORE, mode=MODE)
    
    # set pixel size and time delay
    dqt.pixel_size = pixel_size[num]
    dqt.set_frame_rate(frame_rate_fast)
    
    # compute azimuthal average
    bins = int(len(dqt.ky) / 2)                       # number of bins
    bin_range = (0, dqt.ky[-1])                       # bin range
    ccm = fd.mask.central_cross_mask(dqt.shape[1:])   # mask for central cross
    
    aa_fast = fd.azimuthal_average(dqt, bins=bins, range=bin_range, mask=ccm)
    
    # delete unused variables
    del img_seq, dqt
    gc.collect()
    
    # analyze slow acquisition
    # notify user on progress
    print(f'Analyzing slow movie #{num+1} of {len(file_paths_slow)} ...')
    
    # read image sequence
    img_seq = fd.read_images(fpaths)
    
    # compute structure function
    lags = range(1, int(len(img_seq) / 2))              # we analyze delays only up to half the length of the video
    dqt = fd.ddm(img_seq, lags, core=CORE, mode=MODE)
    
    # set pixel size and time delay
    dqt.pixel_size = pixel_size[num]
    dqt.set_frame_rate(frame_rate_slow)
    
    # compute azimuthal average
    bins = int(len(dqt.ky) / 2)                       # number of bins
    bin_range = (0, dqt.ky[-1])                       # bin range
    ccm = fd.mask.central_cross_mask(dqt.shape[1:])   # mask for central cross
    
    aa_slow = fd.azimuthal_average(dqt, bins=bins, range=bin_range, mask=ccm)
    
    # delete unused variables
    del img_seq, dqt
    gc.collect()
    
    # melt the azimuthal averages
    aa.append(fd.azimuthalaverage.melt(aa_fast, aa_slow))
    
    # delete unused variables
    del aa_fast, aa_slow
    gc.collect();

To ease plotting, we compute also resampled azimuthal averages.

In [ ]:
# resample melt
aa_resampled = []

for a in aa:
    new_taus = fd.lags.logspace_int(a.tau[-1] // a.tau[0], num=200) * a.tau[0]
    aa_resampled.append(a.resample(new_taus))

## 2. Fit the data

### 2.A. Estimate noise and amplitude

To give good starting points to the fit routine, we estimate the amplitude and noise term of the structure function.

In [ ]:
# estimate noise and amplitude terms
Bq = []
Aq = []

for a in aa:
    B_est, _ = fd.noise_est.estimate_camera_noise(a, mode='polyfit', num_points=5)
    Bq.append(B_est) 
    Aq.append(2 * a.var - B_est)

### 2.B. Fit with simple exponential

For particles undergoing Brownian motion, the intermediate scattering function is well-described by a simple exponential function:

$$
f(q, \Delta t) = \exp [- \Gamma(q) \Delta t] ,
$$
where $\Gamma(q) = D q^2$ is the relaxation rate and $D$ is the particles' diffusion coefficient.

We can use the `fastddm.fit_models.simple_exponential_model`, which implements the exponential function in the structure function model:

$$
d(q, \Delta t) = A(q) [1 - \exp(-\Gamma(q) \Delta t)] + B(q) .
$$

In [ ]:
from fastddm.fit import fit_multik
from fastddm.fit_models import simple_exponential_model as model

# choose reference k vector
k_ref = 20

To initiate a fitting process with a reliable starting value for the relaxation rate, we can estimate $\tau = 1/\Gamma$ by first estimating the intermediate scattering function and determining the delay where it has a value of $1/\mathrm{e}$.

In [ ]:
# initialize lists
fit_res = []
model_res = []

for a, B, A in zip(aa, Bq, Aq):
    # estimate intermediate scattering function at `k_ref`
    fqt = 1 - (a.data[k_ref] - B[k_ref]) / A[k_ref]
    
    # estimate Gamma
    tau = a.tau[np.argmin(np.abs(fqt - np.exp(-1)))]
    Gamma0 = 1 / tau
    
    # set model parameters hints
    model.set_param_hint('A', value=A[k_ref])
    model.set_param_hint('B', value=B[k_ref])
    model.set_param_hint('Gamma', value=Gamma0)
    
    # fit
    fr, mr = fit_multik(a, model, k_ref, use_err=True, return_model_results=True)
    fit_res.append(fr)
    model_res.append(mr)

Now, we can visualize the output from the fit. We first display the comparison of the intermediate scattering function $f(\Delta t; q)$ and the best fit, both obtained by inverting the structure function data and best fit with the resulting $A(q)$ and $B(q)$ from the fit

$$
f(q, \Delta t) = 1 - \frac{d(q, \Delta t) - B(q)}{A(q)}
$$

In [ ]:
# select range of wave vector indices
k_min = [0.07, 0.15, 0.44]
k_max = [5, 5, 5] #[4.691, 5.159, 6.2]

In [ ]:
fig = plt.figure()
fig.set_figheight((len(aa) / 2) * fig.get_figheight())
gs = fig.add_gridspec(len(aa), hspace=0)
axs = gs.subplots(sharex=True)

# plot curves
for ax, a, ar, fr, mr, km, kM, l in zip(axs, aa, aa_resampled, fit_res, model_res, k_min, k_max, labels):
    idx_min = np.argwhere(a.k >= km)[0,0]
    idx_max = np.argwhere(a.k <= kM)[-1,0]
    k_list = np.linspace(idx_min, idx_max, num=10, dtype=int)

    cspace = color_space(len(k_list))
    
    for k_idx, c in zip(k_list, cspace):
        # calculate intermediate scattering function
        fqt = 1 - (ar.data[k_idx] - fr['B'][k_idx]) / fr['A'][k_idx]
        fqt_fit = 1 - (mr[k_idx].best_fit - fr['B'][k_idx]) / fr['A'][k_idx]
    
        # plot
        ax.plot(ar.tau, fqt, '.', color=c)
        ax.plot(a.tau, fqt_fit, '-', color=c)
        
        # misc
        ax.set_ylabel(r'$f(\Delta t; q)$')
        ax.set_ylim(-0.1, 1.1)
        # label
        at = AnchoredText(l, prop=dict(size=10), frameon=True, loc='upper right')
        ax.add_artist(at)
        # custom legend
        custom_lines = [
            Line2D([0], [0], marker='.', linestyle='none', color='black', label='data'),
            Line2D([0], [0], linestyle='-', color='black', label='fit'),
        ]
        ax.legend(handles=custom_lines, labelspacing=0.4, loc='lower left')
        
axs[-1].set_xscale('log')
axs[-1].set_xlabel(r'$\Delta t$ (s)')

fig.tight_layout();

### 2.C. Effect of objective magnification on the output

Finally, we show the fit parameters, $\Gamma$, $A$, and $B$.
We also display the lower limit $q_{\text{min}}$ defined in [tutorial1](../Tutorial_1-Particle_sizing/tutorial1.ipynb).

In [ ]:
# optical parameters
L = np.array([256 * ps for ps in pixel_size])
NAo = np.array([0.45, 0.45, 0.7])
lambda0 = 0.53

# limits
q_min = 2 * np.pi / L
q_max = 2 * np.pi * NAo / lambda0
gamma_0 = 1 / aa[0].tau[0]    # also, gamma_0 = frame_rate
gamma_T = 1 / aa[0].tau[-1]

In [ ]:
fig = plt.figure()
fig.set_figheight(1.5 * fig.get_figheight())
gs = fig.add_gridspec(2, hspace=0)
axs = gs.subplots(sharex=True)

# plot
for n, (fr, label) in enumerate(zip(fit_res, labels)):
    # filter only nan and non-successful fits
    tmp_df = fr[fr['success']].dropna()
    
    # display the full data (with transparency)
    axs[0].errorbar(
        tmp_df['k'],
        tmp_df['Gamma'],
        yerr=tmp_df['Gamma_stderr'],
        fmt=f'C{n}o',
        markerfacecolor='none',
        label=label
    )
    axs[1].errorbar(
        tmp_df['k'],
        tmp_df['A'],
        yerr=tmp_df['A_stderr'],
        fmt=f'C{n}o',
        markerfacecolor='none',
    )
    axs[1].plot(
        tmp_df['k'],
        np.full_like(tmp_df['B'], fill_value=tmp_df['B'].mean()),
        f'C{n}--',
    )

# settings
axs[0].set_yscale('log')
axs[0].set_ylabel(r'$\Gamma$ (s$^{-1}$)')
axs[0].legend(loc='upper left')
axs[1].set_xscale('log')
axs[1].set_xlabel(r'$q$ ($\mu$m$^{-1}$)')
axs[1].set_yscale('log')
axs[1].set_ylabel(r'$A$, $B$')
custom_lines = [
    Line2D([0], [0], marker='o', linestyle='none', markerfacecolor='none', color='black', label=r'$A$'),
    Line2D([0], [0], linestyle='--', color='black', label=r'$B$'),
]
axs[1].legend(handles=custom_lines, labelspacing=0.4, loc='upper right')

# plot Gamma limits
axs[0].axhline(gamma_T, color='gray', linestyle='--', alpha=0.5)
axs[0].axhline(gamma_0, color='gray', linestyle='--', alpha=0.5)

# plot q limits
for n, qm in enumerate(q_min):
    axs[0].axvline(x=qm, color=f'C{n}', linestyle='-.', alpha=0.5)

fig.tight_layout();

The vertical dash-dotted lines in the upper panel are the $q_{\text{min}} = 2 \pi / L$ limits for the three objectives.
The effect of the objective magnification (at equal image size in pixels) is to shift the lower boundary of the $q$ range proportionally to the magnification itself.

We can again filter the fit results as we saw in the first tutorial.

In [ ]:
# filter fit results
fit_res_filtered = []

for fr, qm, qM in zip(fit_res, q_min, q_max):
    # remove non-succesful fits and nans
    tmp_df = fr[fr['success']].dropna()
    
    # filter q range
    tmp_df = tmp_df[(tmp_df['k'] >= qm) & (tmp_df['k'] <= qM)]
    
    # filter Gamma range
    tmp_df = tmp_df[(tmp_df['Gamma'] >= gamma_T) & (tmp_df['Gamma'] <= gamma_0)]
    
    # append to list
    fit_res_filtered.append(tmp_df)

In [ ]:
fig = plt.figure()
fig.set_figheight(1.5 * fig.get_figheight())
gs = fig.add_gridspec(2, hspace=0)
axs = gs.subplots(sharex=True)

# plot
for n, (fr, label) in enumerate(zip(fit_res_filtered, labels)):
    
    # display the full data (with transparency)
    axs[0].errorbar(
        fr['k'],
        fr['Gamma'],
        yerr=fr['Gamma_stderr'],
        fmt=f'C{n}o',
        markerfacecolor='none',
        label=label
    )
    axs[1].errorbar(
        fr['k'],
        fr['A'],
        yerr=fr['A_stderr'],
        fmt=f'C{n}o',
        markerfacecolor='none',
    )
    axs[1].plot(
        fr['k'],
        np.full_like(fr['B'], fill_value=fr['B'].mean()),
        f'C{n}--',
    )

# settings
axs[0].set_yscale('log')
axs[0].set_ylabel(r'$\Gamma$ (s$^{-1}$)')
axs[0].legend()
axs[1].set_xscale('log')
axs[1].set_xlabel(r'$q$ ($\mu$m$^{-1}$)')
axs[1].set_yscale('log')
axs[1].set_ylabel(r'$A$, $B$')
custom_lines = [
    Line2D([0], [0], marker='o', linestyle='none', markerfacecolor='none', color='black', label=r'$A$'),
    Line2D([0], [0], linestyle='--', color='black', label=r'$B$'),
]
axs[1].legend(handles=custom_lines, labelspacing=0.4, loc='upper right')

fig.tight_layout();

Notice that the amplitude seems to scale with the square of the objective magnification.
To check this, we analyze measurements performed on the same sample with the first two objectives (10$\times$ and 20$\times$) and using different lamp intensities, measured as the average signal registered by the camera sensor (10k, 25k, and 50k).

The next code cells repeat what we have seen so far, so we will not comment further on them.

In [ ]:
# high frame rate videos file names (including enclosing folder)
file_names_fast2 = [
    'PS_252nm_phi_1e-3_10x_NA_0-45_10k/fast.nd2',
    'PS_252nm_phi_1e-3_10x_NA_0-45_25k/fast.nd2',
    'PS_252nm_phi_1e-3_10x_NA_0-45_50k/fast.nd2',
    'PS_252nm_phi_1e-3_20x_NA_0-45_10k/fast.nd2',
    'PS_252nm_phi_1e-3_20x_NA_0-45_25k/fast.nd2',
    'PS_252nm_phi_1e-3_20x_NA_0-45_50k/fast.nd2',
]

# create full paths
file_paths_fast2 = [os.path.join(main_directory, fn) for fn in file_names_fast2]

# low frame rate videos file names (including enclosing folder)
file_names_slow2 = [
    'PS_252nm_phi_1e-3_10x_NA_0-45_10k/slow.nd2',
    'PS_252nm_phi_1e-3_10x_NA_0-45_25k/slow.nd2',
    'PS_252nm_phi_1e-3_10x_NA_0-45_50k/slow.nd2',
    'PS_252nm_phi_1e-3_20x_NA_0-45_10k/slow.nd2',
    'PS_252nm_phi_1e-3_20x_NA_0-45_25k/slow.nd2',
    'PS_252nm_phi_1e-3_20x_NA_0-45_50k/slow.nd2',
]

# create full paths
file_paths_slow2 = [os.path.join(main_directory, fn) for fn in file_names_slow2]

# list of labels
labels2 = [
    r'10x, 10k',
    r'10x, 25k',
    r'10x, 50k',
    r'20x, 10k',
    r'20x, 25k',
    r'20x, 50k',
]

In [ ]:
fig = plt.figure()
gs = fig.add_gridspec(ncols=3, nrows=2)
axs = gs.subplots()

# read the images
images = [fd.read_images(fp, seq=[0])[0] for fp in file_paths_fast2]
images = np.array(images)

for ax, img, label in zip(axs.flatten(), images, labels2):
    ax.imshow(img, cmap='gray')
    
    ax.set_axis_off()
    ax.set_title(label)

fig.tight_layout();

In [ ]:
print('--- Calculating structure functions and azimuthal averages... ---')

# initialize list of azimuthal averages
aa2 = []

for num, (fpathf, fpaths) in enumerate(zip(file_paths_fast2, file_paths_slow2)):
    # analyze fast acquisition
    # notify user on progress
    print(f'Analyzing fast movie #{num+1} of {len(file_paths_fast2)} ...')
    
    # read image sequence
    img_seq = fd.read_images(fpathf)
    
    # compute structure function
    lags = range(1, int(len(img_seq) / 2))              # we analyze delays only up to half the length of the video
    dqt = fd.ddm(img_seq, lags, core=CORE, mode=MODE)
    
    # set pixel size and time delay
    dqt.pixel_size = pixel_size[num // 3]
    dqt.set_frame_rate(frame_rate_fast)
    
    # compute azimuthal average
    bins = int(len(dqt.ky) / 2)                       # number of bins
    bin_range = (0, dqt.ky[-1])                       # bin range
    ccm = fd.mask.central_cross_mask(dqt.shape[1:])   # mask for central cross
    
    aa_fast = fd.azimuthal_average(dqt, bins=bins, range=bin_range, mask=ccm)
    
    # delete unused variables
    del img_seq, dqt
    gc.collect()
    
    # analyze slow acquisition
    # notify user on progress
    print(f'Analyzing slow movie #{num+1} of {len(file_paths_slow2)} ...')
    
    # read image sequence
    img_seq = fd.read_images(fpaths)
    
    # compute structure function
    lags = range(1, int(len(img_seq) / 2))              # we analyze delays only up to half the length of the video
    dqt = fd.ddm(img_seq, lags, core=CORE, mode=MODE)
    
    # set pixel size and time delay
    dqt.pixel_size = pixel_size[num // 3]
    dqt.set_frame_rate(frame_rate_slow)
    
    # compute azimuthal average
    bins = int(len(dqt.ky) / 2)                       # number of bins
    bin_range = (0, dqt.ky[-1])                       # bin range
    ccm = fd.mask.central_cross_mask(dqt.shape[1:])   # mask for central cross
    
    aa_slow = fd.azimuthal_average(dqt, bins=bins, range=bin_range, mask=ccm)
    
    # delete unused variables
    del img_seq, dqt
    gc.collect()
    
    # melt the azimuthal averages
    aa2.append(fd.azimuthalaverage.melt(aa_fast, aa_slow))
    
    # delete unused variables
    del aa_fast, aa_slow
    gc.collect();

In [ ]:
# resample melt
aa_resampled2 = []

for a in aa2:
    new_taus = fd.lags.logspace_int(a.tau[-1] // a.tau[0], num=200) * a.tau[0]
    aa_resampled2.append(a.resample(new_taus))

In [ ]:
# estimate noise and amplitude terms
Bq2 = []
Aq2 = []

for a in aa2:
    B_est, _ = fd.noise_est.estimate_camera_noise(a, mode='polyfit', num_points=5)
    Bq2.append(B_est) 
    Aq2.append(2 * a.var - B_est)

In [ ]:
# fit data
# initialize lists
fit_res2 = []
model_res2 = []

for a, B, A in zip(aa2, Bq2, Aq2):
    # estimate intermediate scattering function at `k_ref`
    fqt = 1 - (a.data[k_ref] - B[k_ref]) / A[k_ref]
    
    # estimate Gamma
    tau = a.tau[np.argmin(np.abs(fqt - np.exp(-1)))]
    Gamma0 = 1 / tau
    
    # set model parameters hints
    model.set_param_hint('A', value=A[k_ref])
    model.set_param_hint('B', value=B[k_ref])
    model.set_param_hint('Gamma', value=Gamma0)
    
    # fit
    fr, mr = fit_multik(a, model, k_ref, use_err=True, return_model_results=True)
    fit_res2.append(fr)
    model_res2.append(mr)

In [ ]:
fig = plt.figure()
fig.set_figheight(1.5 * fig.get_figheight())
gs = fig.add_gridspec(2, hspace=0)
axs = gs.subplots(sharex=True)

# plot
for n, (fr, label) in enumerate(zip(fit_res2, labels2)):
    # filter only nan and non-successful fits
    tmp_df = fr[fr['success']].dropna()
    
    # display the full data (with transparency)
    axs[0].errorbar(
        tmp_df['k'],
        tmp_df['Gamma'],
        yerr=tmp_df['Gamma_stderr'],
        fmt=f'C{n}o',
        markerfacecolor='none',
        label=label
    )
    axs[1].errorbar(
        tmp_df['k'],
        tmp_df['A'],
        yerr=tmp_df['A_stderr'],
        fmt=f'C{n}o',
        markerfacecolor='none',
    )
    axs[1].plot(
        tmp_df['k'],
        np.full_like(tmp_df['B'], fill_value=tmp_df['B'].mean()),
        f'C{n}--',
    )

# settings
axs[0].set_yscale('log')
axs[0].set_ylabel(r'$\Gamma$ (s$^{-1}$)')
axs[0].legend(loc='upper left')
axs[1].set_xscale('log')
axs[1].set_xlabel(r'$q$ ($\mu$m$^{-1}$)')
axs[1].set_yscale('log')
axs[1].set_ylabel(r'$A$, $B$')
custom_lines = [
    Line2D([0], [0], marker='o', linestyle='none', markerfacecolor='none', color='black', label=r'$A$'),
    Line2D([0], [0], linestyle='--', color='black', label=r'$B$'),
]
axs[1].legend(handles=custom_lines, labelspacing=0.4, loc='upper right')

# plot Gamma limits
axs[0].axhline(gamma_T, color='gray', linestyle='--', alpha=0.5)
axs[0].axhline(gamma_0, color='gray', linestyle='--', alpha=0.5)

# plot q limits
for n, qm in enumerate(q_min[:2]):
    axs[0].axvline(x=qm, color=f'C{n*3}', linestyle='-.', alpha=0.5)

fig.tight_layout();